In [119]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectPercentile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import Imputer

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [121]:
from subprocess import check_output


In [129]:
class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=5.0, impute=True, impute_strategy='median'):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
        # The statsmodel function will fail with NaN values, as such we have to impute them.
        # By default we impute using the median value.
        # This imputation could be taken out and added as part of an sklearn Pipeline.
        if impute:
            self.imputer = Imputer(strategy=impute_strategy)

    def fit(self, X, y=None):
        print('ReduceVIF fit')
        if hasattr(self, 'imputer'):
            self.imputer.fit(X)
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        if hasattr(self, 'imputer'):
            X = pd.DataFrame(self.imputer.transform(X), columns=columns)
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=5.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            # Loop repeatedly until we find that all columns within our dataset
            # have a VIF value we're happy with.
            variables = X.columns
            dropped=False
            vif = []
            new_vif = 0
            for var in X.columns:
                new_vif = variance_inflation_factor(X[variables].values, X.columns.get_loc(var))
                vif.append(new_vif)
                if np.isinf(new_vif):
                    break
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print(f'Dropping {X.columns[maxloc]} with vif={max_vif}')
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X
    
transformer = ReduceVIF()

In [130]:
df = pd.read_csv(r'B:\Phillip\Mapping\BlockGroupJoin\CNTYVARS.csv')
df['HSGRAD_PRCNT'] = df['HSGRAD_CY']/df['TOTPOP_CY']

In [131]:
dtype_df = df.dtypes.reset_index()
dtype_df.columns = ["Count", "Column Type"]
print(dtype_df[dtype_df['Column Type'] == 'object'])

         Count Column Type
783   TSEGCODE      object
784   TSEGNAME      object
854  TLIFECODE      object
855  TLIFENAME      object


In [ ]:
dp = ['OBJECTID','Customers', 'UNITS', 'SALES', 'LOG_UNITS', 'TSEGCODE','TSEGNAME','TLIFECODE','TLIFENAME']

X = df.drop(dp, axis=1)

X2 = transformer.fit_transform(X, y)

X2.head()

ReduceVIF fit
ReduceVIF transform


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Dropping TOTPOP_CY with vif=inf
Dropping HHPOP_CY with vif=inf
Dropping TOTHH_CY with vif=inf
Dropping TOTHU_CY with vif=inf
Dropping OWNER_CY with vif=inf
Dropping POP0_CY with vif=inf
Dropping POP5_CY with vif=inf
Dropping POP10_CY with vif=inf
Dropping POP15_CY with vif=inf
Dropping POP20_CY with vif=inf
Dropping POP25_CY with vif=inf
Dropping POP30_CY with vif=inf
Dropping POP35_CY with vif=inf
Dropping POP40_CY with vif=inf
Dropping POP45_CY with vif=inf
Dropping POP50_CY with vif=inf
Dropping POP55_CY with vif=inf
Dropping POP60_CY with vif=inf
Dropping POP65_CY with vif=inf
Dropping POP70_CY with vif=inf
Dropping POP75_CY with vif=inf
Dropping POP80_CY with vif=inf
Dropping POP85_CY with vif=inf
Dropping POP18UP_CY with vif=inf


In [113]:
y = df[['LOG_UNITS']]
y.shape

(3109, 1)

In [115]:
selector = SelectPercentile(mutual_info_regression, percentile=10)
selector.fit(X, y)
XNew = selector.fit_transform(X, y)

lst = list()
for n, s in zip(X.columns, selector.scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['score','name'])
    


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [117]:
scores.sort_values(by='score',inplace=True, ascending=False)
scores.reset_index(inplace=True)
scores.to_csv(r'B:\Phillip\Mapping\BlockGroupJoin\varscores.csv')

In [89]:
XNew.shape
XNew.

(3109, 117)

In [56]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

In [80]:
regression = LinearRegression()
selector = RFECV(estimator=regression, cv=2, scoring='neg_mean_squared_error')
selector.fit(X, y)
print('Optimal number of features: %d' % selector.n_features_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimal number of features: 13


In [90]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.show()

AttributeError: 'SelectPercentile' object has no attribute 'grid_scores_'

In [78]:
lst = list()
for n, s in zip(X.columns, selector.grid_scores_):
    lst.append((s, n))

scores = pd.DataFrame(lst, columns=['score','name'])

In [79]:
scores.sort_values(by='score',inplace=True, ascending=False)
scores.reset_index(inplace=True)
scores


index         score        name
0        15 -1.238624e+00  FAMGRW10CY
1        17 -1.240941e+00     POP5_CY
2        18 -1.244718e+00    POP10_CY
3        16 -1.248249e+00     POP0_CY
4        21 -1.259206e+00    POP25_CY
5        19 -1.271689e+00    POP15_CY
6        20 -1.272744e+00    POP20_CY
7        22 -1.276147e+00    POP30_CY
8        14 -1.302547e+00   HHGRW10CY
9        12 -1.309250e+00   VACANT_CY
10       13 -1.309521e+00  POPGRW10CY
11       23 -1.322585e+00    POP35_CY
12       25 -1.335920e+00    POP45_CY
13       24 -1.386597e+00    POP40_CY
14       26 -1.421899e+00    POP50_CY
15        7 -1.442899e+00    FAMHH_CY
16       28 -1.450098e+00    POP60_CY
17       27 -1.483347e+00    POP55_CY
18       36 -1.501105e+00   MEDAGE_CY
19       34 -1.509022e+00  POP18UP_CY
20       35 -1.529115e+00  POP21UP_CY
21       11 -1.551548e+00   RENTER_CY
22       29 -1.552728e+00    POP65_CY
23       33 -1.553443e+00    POP85_CY
24       37 -1.571530e+00    MALES_CY
25       39 -1.585778e+00    MALE5_CY
26       38 -1.585934e+00    MALE0_CY
27       10 -1.606330e+00    OWNER_CY
28       30 -1.629557e+00    POP70_CY
29       40 -1.636244e+00   MALE10_CY
...     ...           ...         ...
1133    822 -1.419216e+11       THH40
1134    821 -1.419218e+11       THH39
1135    823 -1.423137e+11       THH41
1136    824 -1.423138e+11       THH42
1137    825 -1.423222e+11       THH43
1138    739 -1.505638e+11  AGGIA45_FY
1139    736 -1.524148e+11  AGGIA15_FY
1140    737 -1.524177e+11  AGGIA25_FY
1141    738 -1.524183e+11  AGGIA35_FY
1142    735 -1.532210e+11  AGGIA75_CY
1143    734 -1.533240e+11  AGGIA65_CY
1144    742 -1.533346e+11  AGGIA75_FY
1145    740 -1.533348e+11  AGGIA55_FY
1146    741 -1.533352e+11  AGGIA65_FY
1147    729 -1.585846e+11  AGGIA15_CY
1148    730 -1.585847e+11  AGGIA25_CY
1149    732 -1.680418e+11  AGGIA45_CY
1150    733 -1.680437e+11  AGGIA55_CY
1151    728 -1.728170e+11  AGGHINC_FY
1152    726 -1.728173e+11   AGGINC_FY
1153    727 -1.728177e+11  AGGHINC_CY
1154    731 -1.735612e+11  AGGIA35_CY
1155    720 -1.901369e+11  HAGEBASECY
1156    721 -1.901374e+11  HSPMBASECY
1157    718 -1.901822e+11  MLTMBASECY
1158    719 -1.901835e+11  MLTFBASECY
1159    724 -1.939082e+11  OCCBASE_CY
1160    725 -1.939088e+11   AGGINC_CY
1161    722 -1.939487e+11  HSPFBASECY
1162    723 -1.939518e+11  INDBASE_CY

[1163 rows x 3 columns]